In [2]:
!pip install mysql --quiet

     |████████████████████████████████| 88 kB 3.7 MB/s 


In [3]:
import mysql
import mysql.connector as msql
from mysql.connector import Error

import pandas as pd


ModuleNotFoundError: ignored

In [ ]:
worldnews = pd.read_csv('worldnews.csv')
worldnews.head()

,Unnamed: 0,fullname,title,article,author,domain,date_created,num_comments,post_score,total_awards,...,top_comment_score,second_comment,second_comment_score,third_comment,third_comment_score,neg,neu,pos,compound,label
0,0,t3_k4qide,An anti-gay Hungarian politician has resigned ...,https://www.businessinsider.com/hungarian-mep-...,stem12345679,businessinsider.com,2020-12-01 10:15:32,8437,204543,0,...,33800,25 men is a lot of men for a homophobe,26571,> A source in the local police told the newspa...,20222,0.118,0.882,0.000,-0.2500,neg
1,1,t3_eclwg9,Trump Impeached for Abuse of Power,https://www.nytimes.com/interactive/2019/12/18...,MachoNachoTaco,nytimes.com,2019-12-18 17:23:35,20128,202901,0,...,25989,Friendly reminder to vote in the next election...,20403,Okay for everyone wondering what impeachment i...,15940,0.457,0.543,0.000,-0.6369,neg
2,2,t3_t3pgaz,Vladimir Putin's black belt revoked by interna...,https://www.newsweek.com/vladimir-putins-black...,bichonista,newsweek.com,2022-02-28 12:45:18,6961,200153,0,...,37491,The sanctions are getting fucking brutal,36525,I'm sorry I know the situation is serious but ...,10933,0.000,1.000,0.000,0.0000,neu
3,3,t3_901p5f,"Two weeks before his inauguration, Donald J. T...",https://www.nytimes.com/2018/07/18/world/europ...,DoremusJessup,nytimes.com,2018-07-18 19:06:35,18086,189349,0,...,25333,It still baffles me this is real life.,16727,Upvote this shit to the top so everyone knows ...,14605,0.000,0.902,0.098,0.5233,pos
4,4,t3_x96k3v,"Queen Elizabeth II has died, Buckingham Palace...",https://www.bbc.co.uk/news/uk-61585886,pipsdontsqueak,bbc.co.uk,2022-09-08 10:32:06,16767,188705,0,...,20625,It is so weird hearing the BBC reporters talki...,20507,"Queen's first PM was Churchill, who was born 1...",18838,0.340,0.660,0.000,-0.5574,neg


In [ ]:
conn = msql.connect(
  host="localhost",
  user="root",
  password="12Tasha12!",
  database = "FINAL"
)

In [ ]:
print(conn)


#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [ ]:
cursor.execute('''DROP INDEX domain_index ON post;''')


In [ ]:
import mysql
import mysql.connector as msql
from mysql.connector import Error
conn = msql.connect(
  host="localhost",
  user="root",
  password="12Tasha12!",
  database = "FINAL"
)
if conn.is_connected():
    cursor = conn.cursor(buffered=True)
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)
    cursor.execute('DROP TABLE IF EXISTS post;')
    cursor.execute('DROP TABLE IF EXISTS engagement;')
    cursor.execute('DROP TABLE IF EXISTS top_comments;')
    cursor.execute('DROP TABLE IF EXISTS comments;')
    cursor.execute('DROP TABLE IF EXISTS sentiment;')
    print('Creating table....')
# in the below line please pass the create table statement which you want #to create
    cursor.execute(""" CREATE TABLE IF NOT EXISTS sentiment (
                                        sentiment_id int PRIMARY KEY AUTO_INCREMENT,
                                        neg_score decimal (5,4),
                                        neu_score decimal (5,4),
                                        pos_score decimal (5,4),
                                        compund_score decimal (5,4),
                                        label text
                                        );""")
    cursor.execute(""" CREATE TABLE IF NOT EXISTS top_comments (
                                    top_comments_id int PRIMARY KEY AUTO_INCREMENT,
                                    text_1 text,
                                    score_1 int,                                  
                                    text_2 text,
                                    score_2 int,                                 
                                    text_3 text,
                                    score_3 int
                                    );""")                                    
    cursor.execute(""" CREATE TABLE IF NOT EXISTS engagement (
                                        engagement_id int PRIMARY KEY AUTO_INCREMENT,
                                        num_comments int,
                                        upvotes int,
                                        total_awards int,
                                        top_comments_id int,
                                        FOREIGN KEY (top_comments_id) REFERENCES top_comments(top_comments_id) ON DELETE CASCADE
                                    ); """)
    cursor.execute(""" CREATE TABLE IF NOT EXISTS post (
                                    post_ID VARCHAR(255) PRIMARY KEY,
                                    title text,
                                    article_url text,
                                    domain text,
                                    date_created DATETIME,
                                    engagement_id int,
                                    sentiment_id int,
                                    FOREIGN KEY (engagement_id) REFERENCES engagement(engagement_id) ON DELETE CASCADE,
                                    FOREIGN KEY (sentiment_id) REFERENCES sentiment(sentiment_id) ON DELETE CASCADE
                                ); """)

You're connected to database:  ('final',)
Creating table....


In [ ]:
i = 1
for row in worldnews.itertuples():
    cursor.execute('''
                INSERT INTO sentiment (neg_score, neu_score, pos_score, compund_score, label)
                VALUES (%s,%s,%s,%s,%s);
                ''',
                (row.neg, 
                row.neu,
                row.pos,
                row.compound,
                row.label)
                )
    cursor.execute('''
                INSERT INTO top_comments (text_1, score_1, text_2, score_2, text_3, score_3)
                VALUES (%s,%s,%s,%s,%s,%s);
                ''',
                (row.top_comment,
                row.top_comment_score,
                row.second_comment,
                row.second_comment_score,
                row.third_comment,
                row.third_comment_score)
                )
    cursor.execute('''
                INSERT INTO engagement (num_comments, upvotes, total_awards, top_comments_id)
                VALUES (%s,%s,%s,%s);
                ''',
                (row.num_comments,
                row.post_score,
                row.total_awards,
                i)
                )
    cursor.execute('''
                INSERT INTO post (post_ID,title, article_url, domain, date_created, engagement_id, sentiment_id)
                VALUES (%s,%s,%s,%s,%s,%s,%s);
                ''',
                (row.fullname,
                row.title,
                row.article,
                row.domain,
                row.date_created,
                i,
                i)
                )
    i += 1
conn.commit()

In [ ]:
cursor.execute('''  CREATE TRIGGER insert_fk AFTER INSERT ON post
                    FOR EACH ROW
                    BEGIN
                    INSERT INTO engagement (top_comments_id) VALUES (NEW.top_comments_id);
                    INSERT INTO post (engagement_id,sentiment_id) VALUES (NEW.engagement_id, NEW.sentiment_id);
''')

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 5

In [ ]:
cursor.execute('''SELECT * FROM post
                INNER JOIN engagement ON post.engagement_id = engagement.engagement_id
                INNER JOIN sentiment ON post.sentiment_id = sentiment.sentiment_id
                INNER JOIN top_comments ON engagement.top_comments_id = top_comments.top_comments_id
                ''')
result = cursor.fetchall()

In [ ]:
for res in result:
    print(res)

('t3_4d75i7', '2.6 terabyte leak of Panamanian shell company data reveals "how a global industry led by major banks, legal firms, and asset management companies secretly manages the estates of politicians, Fifa officials, fraudsters and drug smugglers, celebrities and professional athletes."', 'http://panamapapers.sueddeutsche.de/articles/56febff0a1bb8d3c3495adf4/', 'panamapapers.sueddeutsche.de', datetime.datetime(2016, 4, 3, 11, 1, 46), 10, 10, 10, 12069, 154760, 0, 10, 10, Decimal('0.1810'), Decimal('0.7300'), Decimal('0.0880'), Decimal('-0.6369'), 'neg', 10, '**edit 7: [live thread](https://www.reddit.com/live/wp1fvdxxwb45/)**\n\nOver 100 news agencies sitting in 100 contries all over the world are disclosing the panama papers tonight at the exact same time. This will be a huge hit for several large figures in world sports, politics and economy. Names like Lionel messi, Vladimir Putin and the prime minister of Iceland are already leaked. I am curious to see who turns up as well. Wh